In [1]:
from pathlib import Path
from urllib.request import urlretrieve
import zipfile
from matplotlib.pyplot import imread
from scipy.ndimage.interpolation import zoom#
import numpy as np


def fetch_cropped_yaleb(data_folder, zooming=0.5, max_n_subjects=None):
    """Returns a dictionary of paths
    
    Parameters
    ----------
    data_folder: string
    zooming: float, optional, default is 0.5
        factor by which to resize the images
    max_n_subjects: {None, int}, optional, default is None
        if not None, only the first max_n_subjects are returned
    
    Returns
    -------
    dict: {
        subjects_1: {'images': [image_1, ... image_N],
               'ambient': image_ambient,
        }
    }
    
    images are stored as numpy arrays
    """
    url = 'http://vision.ucsd.edu/extyaleb/CroppedYaleBZip/CroppedYale.zip'
    yaleb_path = Path(data_folder).joinpath('cropped_yaleb')
    
    if not yaleb_path.joinpath('CroppedYale').exists():
        yaleb_path.mkdir(parents=True)
    
    # If not already unzip, do it
    if not list(yaleb_path.iterdir()):
        zip_path = yaleb_path.joinpath('yaleb.zip')
        
        # If zip not already downloaded, download it
        if not zip_path.exists():
            urlretrieve(url, zip_path.as_posix())
        
        zfile = zipfile.ZipFile(zip_path.as_posix())
        zfile.extractall(path=yaleb_path.as_posix())

    yaleb = {}
    for folder_path in yaleb_path.joinpath('CroppedYale').iterdir():
        if max_n_subjects is not None and len(yaleb) > max_n_subjects:
            return yaleb
        
        if not folder_path.is_dir():
            continue
            
        video_name = folder_path.name
        paths = sorted(list(folder_path.glob('*.pgm')))
        images = []
        for path in paths:
            if 'Ambient' in path.name:
                ambient = imread(path.as_posix())
            else:
                images.append(zoom(imread(path.as_posix()), zooming)[None, ...])
                
        data = {'images':np.concatenate(images),
        'ambient':ambient}
        yaleb[video_name] = data

    return yaleb

In [2]:
dataset_path = '/data/tensorly_data/'

data = fetch_cropped_yaleb(dataset_path, zooming=0.3, max_n_subjects=5)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: '\\data\\tensorly_data\\cropped_yaleb'

In [ ]:
data

In [ ]:
data.keys()

In [ ]:
data['yaleB01']['ambient'].shape

In [ ]:
data['yaleB01']['images'].shape

In [ ]:
X = np.concatenate([data[key]['images'] for key in data], axis=0)

print(X.shape)

In [ ]:
[(key, data[key]['images'][0,0,0]) for key in data] #play with list comprehension

In [ ]:
def fetch_cropped_yaleb_testing(data_folder, zooming=0.5, max_n_subjects=None):
    
    yaleb_path = Path(data_folder).joinpath('cropped_yaleb')
  
    for folder_path in yaleb_path.joinpath('CroppedYale').iterdir():
         
        if not folder_path.is_dir():
            continue
            
        video_name = folder_path.name
        print(video_name)

    return 0

In [ ]:
fetch_cropped_yaleb_testing(dataset_path, zooming=0.3, max_n_subjects=5)

In [ ]:
X = X.astype(np.float64)
X -= X.mean()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def visualise_images(X, n_images, n_columns, randomise=True):
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    indices = indices[:n_images]
    cmap = plt.cm.Greys_r
    n_rows = np.ceil(n_images / n_columns).astype(int)
    fig = plt.figure(figsize=(2*n_columns, 2*n_rows))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

    # plot the digits: each image is 8x8 pixels
    for i, e in enumerate(indices):
        ax = fig.add_subplot(n_rows, n_columns, i + 1, xticks=[], yticks=[])
        ax.imshow(X[e], cmap=cmap, interpolation='nearest')

In [ ]:
visualise_images(X, 12, 4)

In [ ]:
import random
 
def add_noise(X, percent=0.15):
    num_images, row, col = X.shape
    
    n1 = int(row*col*percent)
    for i in range(num_images):
        
        fmin, fmax = np.min(X[i]), np.max(X[i])
        
        for j in range(n1):
    
            y_coord=random.randint(0, row - 1)
            x_coord=random.randint(0, col - 1)
            scale = random.randint(0, 100)/100 #random between 0 and 1
            val = fmin + scale*(fmax - fmin) #random betweeen fmin and fmax
            X[i,y_coord, x_coord]=val

In [ ]:
add_noise(X)

In [ ]:
visualise_images(X, 12, 4)